In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
import os
import boto3
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import csv
import numpy as np
from pathlib import Path

In [3]:
def maybe_pickle(data, filename, force=False):
    if os.path.exists(filename) and not force:
        # You may override by setting force=True.
        print('%s already present - Skipping pickling.' % filename)
    else:
        print('Pickling %s.' % filename)
        try:
            with open(filename, 'wb') as f:
                pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to', filename, ':', e)


def maybe_download(table, force=False):
    storage_file = table + ".pkl"
    if force or not os.path.exists(storage_file):
        data = data_retrieve(table)
        maybe_pickle(data, storage_file, force)
    return storage_file
        
def data_retrieve(table, page_size=100):
    """download data from table"""
    client = boto3.client('dynamodb')
    paginator = client.get_paginator('scan')

    counter = 0
    items = []
    for page in paginator.paginate(TableName=table, PaginationConfig={"PageSize": page_size}):
        items.extend(page["Items"])
        
        counter += 1
        if counter % 50 == 0:  # print debug every 50 pages
            print("items: {}. next page. {}. Count {}. ScannedCount: {}".format(len(items), counter, page["Count"], page["ScannedCount"]))


    return items
    

In [4]:
# data_file = maybe_download("apthunt")
data_file = maybe_download("apthunt", force=True)
data = pickle.load(open(data_file, 'rb'))

items: 5000. next page. 50. Count 100. ScannedCount: 100
items: 10000. next page. 100. Count 100. ScannedCount: 100
items: 15000. next page. 150. Count 100. ScannedCount: 100
items: 20000. next page. 200. Count 100. ScannedCount: 100
items: 25000. next page. 250. Count 100. ScannedCount: 100
items: 30000. next page. 300. Count 100. ScannedCount: 100
items: 35000. next page. 350. Count 100. ScannedCount: 100
items: 40000. next page. 400. Count 100. ScannedCount: 100
items: 45000. next page. 450. Count 100. ScannedCount: 100
items: 50000. next page. 500. Count 100. ScannedCount: 100
items: 55000. next page. 550. Count 100. ScannedCount: 100
items: 60000. next page. 600. Count 100. ScannedCount: 100
items: 65000. next page. 650. Count 100. ScannedCount: 100
items: 70000. next page. 700. Count 100. ScannedCount: 100
items: 75000. next page. 750. Count 100. ScannedCount: 100
items: 80000. next page. 800. Count 100. ScannedCount: 100
items: 85000. next page. 850. Count 100. ScannedCount: 100

items: 680000. next page. 6800. Count 100. ScannedCount: 100
items: 685000. next page. 6850. Count 100. ScannedCount: 100
items: 690000. next page. 6900. Count 100. ScannedCount: 100
items: 695000. next page. 6950. Count 100. ScannedCount: 100
items: 700000. next page. 7000. Count 100. ScannedCount: 100
items: 705000. next page. 7050. Count 100. ScannedCount: 100
items: 710000. next page. 7100. Count 100. ScannedCount: 100
items: 715000. next page. 7150. Count 100. ScannedCount: 100
items: 720000. next page. 7200. Count 100. ScannedCount: 100
items: 725000. next page. 7250. Count 100. ScannedCount: 100
items: 730000. next page. 7300. Count 100. ScannedCount: 100
items: 735000. next page. 7350. Count 100. ScannedCount: 100
items: 740000. next page. 7400. Count 100. ScannedCount: 100
items: 745000. next page. 7450. Count 100. ScannedCount: 100
items: 750000. next page. 7500. Count 100. ScannedCount: 100
items: 755000. next page. 7550. Count 100. ScannedCount: 100
items: 760000. next page

In [5]:
print("wow")

wow


In [6]:
# To go from dynamo format to python
from boto3.dynamodb import types

deserializer = types.TypeDeserializer()
python_data = [{k: deserializer.deserialize(v) for k,v in d.items()} for d in data]

In [7]:
del data

In [8]:
df = pd.DataFrame(python_data)

In [9]:
del python_data

In [ ]:
with open("df.pkl", 'wb') as f:
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)


## can restore from here

In [ ]:
df = pickle.load(open("df.pkl", 'rb'))

In [ ]:
df.head()

In [ ]:
# small cleanup of "simulated"
del df["FeedTitle"]
del df["FeedUrl"]
del df["PostContent"]
del df["PostPublished"]
del df["PostTitle"]
# delete parsed_notices. looks like it is the same
del df["parsed_notices"]
# no need for price text
del df["parsed_price_text"]

In [ ]:
df.columns

In [ ]:
def clean_parsed_args(args):
    if not isinstance(args, list):
        return []
    clean = []
    for p in args:
        # remove days
        if p.startswith("friday") or p.startswith("monday") or p.startswith("saturday") or p.startswith("sunday") or p.startswith("thursday") or p.startswith("tuesday") or p.startswith("wednesday"):
            continue
        # remove br/ba
        if "BR / " in p: continue
        # remove ft2
        if p.endswith("ft2"): continue

        # remove application fee
        if p.startswith("application fee"): continue
        
        # remove broker fee
        if p.startswith("broker fee"): continue
        
        # remove available
        if p.startswith("available"): continue
        
        # remove listed by
        if p.startswith("listed by"): continue
        clean.append(p)
    return clean
        

In [ ]:
df['parsed_attrs'] = df['parsed_attrs'].map(clean_parsed_args)
# one_hot = pd.get_dummies(df['parsed_attrs'])

In [ ]:
df['parsed_attrs']

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
def get_type(attrs):
    if not isinstance(attrs, list):
        return None
    types = set([
        "apartment", 
        "townhouse", 
        "loft", 
        "land", 
        "house", 
        "duplex", 
        "flat", 
        "condo", 
        "cottage/cabin"
    ])
    return ",".join(sorted(types & set(attrs)))

In [ ]:
def prepare_data(df):
    new  = pd.DataFrame()
    new["PostUrl"] = df["PostUrl"]
    new["latitude"] = df["parsed_data_latitude"]
    new["longitude"] = df["parsed_data_longitude"]
    new["district"] = df["parsed_district"].map(lambda s: s.strip("()") if isinstance(s, str) else s)
    new["address"] = df["parsed_map_address"].map(lambda x: x if x != "(google map)" else None)
    new["housing"] = df["parsed_housing"]
    
    new["price"] = df["parsed_price"].map(lambda x: float(x) if x else x)
    
    # to numerical values
    new['price'] = pd.to_numeric(new['price'])
    new['longitude'] = pd.to_numeric(new['longitude'])
    new['latitude'] = pd.to_numeric(new['latitude'])
    
    new['bedrooms'] = new['housing'].str.extract('(\d+)br\s.*', expand=True)
    new['bedrooms'] = pd.to_numeric(new['bedrooms'])
    
    new['area'] = new['housing'].str.extract('(\d+)ft2', expand=True)
    new['area'] = pd.to_numeric(new['area'])
    
    new['type'] = df["parsed_attrs"].map(get_type)
    new['catsok'] = df["parsed_attrs"].map(lambda x: "cats are OK - purrr" in x)
    new['dogsok'] = df["parsed_attrs"].map(lambda x: "dogs are OK - wooof" in x)
    new['garagea'] = df["parsed_attrs"].map(lambda x: "attached garage" in x)
    new['garaged'] = df["parsed_attrs"].map(lambda x: "detached garage" in x)
    new['furnished'] = df["parsed_attrs"].map(lambda x: "furnished" in x)
    
    new['laundryb'] = df["parsed_attrs"].map(lambda x: "laundry in bldg" in x)
    new['laundrys'] = df["parsed_attrs"].map(lambda x: "laundry on site" in x)
    
    new['wd'] = df["parsed_attrs"].map(lambda x: "w/d in unit" in x)
    new['nthumbs'] = df["parsed_thumbs"].apply(lambda x: len(x) if isinstance(x, list) else 0)

    return new

In [ ]:
short = prepare_data(df)

In [ ]:
short

In [ ]:
short.shape

In [ ]:
short.dropna(subset=["price"], inplace=True)
short.shape

In [ ]:
short["price"].describe()

In [ ]:
# change all prices more than 9k to to 9 k
short.loc[short['price'] > 9000, "price"] = 9000

In [ ]:
short["price"].describe()

In [ ]:
short.shape

In [ ]:
short.drop_duplicates(inplace=True)
short.shape

In [ ]:
short['price_bucket'] = short['price'].apply(lambda x: f"{x//500:02.0f}x500")

In [ ]:
short.head()

In [ ]:
short["price_bucket"].describe()

In [ ]:
export_path = Path("./apthuntdata")
export_path.mkdir(exist_ok=True)
short.to_csv(export_path/"data.csv", index=False) 

In [ ]:
short["price"].describe()

In [ ]:
sns.distplot(short.loc[:, "price"])

In [ ]:
sns.distplot(short.loc[short["price"] < 20000, "price"])

In [ ]:
short[short.price > 15000].describe()

In [ ]:
1094/578376

In [ ]:
from fastai2.tabular.all import *

In [ ]:
def mean_abs_diff(inp, target):
    return (inp - target).abs().mean()
def min_abs_diff(inp, target):
    return (inp - target).abs().min()
def max_abs_diff(inp, target):
    return (inp - target).abs().max()

In [ ]:
learn_inf = load_learner('cltab.pkl')

In [ ]:
short.iloc[0]

In [ ]:
tp = pd.read_csv("apthuntdata/data.csv").loc[0]

In [ ]:
del tp["PostUrl"]
del tp["address"]
del tp["price"]
del tp["price_bucket"]

In [ ]:
tp

In [ ]:
prediction = learn_inf.predict(tp)
i = 0
for p in prediction:
    print("-" * 11)
    print(i)
    i += 1
    print(p)
    print(type(p))

In [ ]:
pd.read_csv("apthuntdata/data.csv").loc[0]

In [ ]:
topred = pd.read_csv("apthuntdata/data.csv")
del topred["PostUrl"]
del topred["address"]
del topred["price"]
del topred["price_bucket"]

In [ ]:
learn_inf.predict(topred)

In [ ]:
topred.loc[0:1]

In [ ]:
topred.apply(lambda x: f"{x['latitude']} + {x['district']}", axis=1)

In [ ]:
topred["price"] = topred.apply(learn_inf.predict, axis=1)